In [ ]:
library(malbacR)
library(pmartR)
library(ggplot2)
library(dplyr)

In [ ]:
data_raw <- readRDS('/mnt/data3/data_exp_0826/result/data_raw.rds')

In [ ]:
f_data <- data_raw$f_data %>% 
    arrange(sample_order)
e_data <- data_raw$e_data %>% 
    select(c('Molecule',f_data$Sample))
pmart_amide <- as.metabData(
  e_data = e_data,
  f_data = f_data,
  e_meta = data_raw$e_meta,
  edata_cname = 'Molecule',
  emeta_cname = 'Molecule',
  fdata_cname = 'Sample'
)
pmart_amide <- edata_replace(pmart_amide, x = 0, y = NA, threshold = NULL)
pmart_amide <- edata_transform(pmart_amide,"log2")
impObj <- imputation(omicsData = pmart_amide)
amide_imp <- apply_imputation(imputeData = impObj, omicsData = pmart_amide)
amide_imp_abund <- edata_transform(amide_imp,"abundance")
amide_imp_abund <- group_designation(amide_imp_abund,main_effects = "batch")

In [ ]:
pmart_amide$f_data

In [ ]:
amide_qcrfsc_abundance <- bc_qcrfsc(
    omicsData = amide_imp_abund,
    qc_cname = "class",
    qc_val = "QC",
    order_cname = "sample_order",
    group_cname = "batch", 
    ntree = 500,
    keep_qc = FALSE
)

In [ ]:
amide_qcrfsc <- edata_transform(omicsData = amide_qcrfsc_abundance,data_scale = "log2")

In [ ]:
saveRDS(amide_qcrfsc,'/mnt/data3/data_exp_0826/result/amide_qcrfsc.rds')

In [ ]:
tmp <- amide_qcrfsc$f_data
tmp[tmp$group_lv2=='QC',]

In [ ]:
set.seed(12345678)
data_plot <- dim_reduction(omicsData = amide_qcrfsc) %>% do.call(rbind,.) %>% t() %>% as.data.frame() %>% 
    mutate(
        PC1 = PC1 %>% as.numeric(),
        PC2 = PC2 %>% as.numeric()
    ) %>% 
    rename(Sample = SampleID) %>% 
    left_join(amide_qcrfsc$f_data,by = 'Sample')
data_plot[data_plot$PC2 < -2,]

In [ ]:
data_meta <- amide_qcrfsc$f_data
data_meta[data_meta$Sample %in% (data_plot$SampleID[data_plot$PC2 < -2]),]

In [ ]:
?geom_text_repel

In [ ]:
data_plot[data_plot$PC2 < -2,]

In [ ]:
library(paletteer)
library(ggrepel)
options(repr.plot.width = 14,repr.plot.height = 10)
# plot(dim_reduction(omicsData = amide_qcrfsc),omicsData = amide_qcrfsc,color_by = "group_lv1")
ggplot(data = data_plot,aes(x = PC1,y = PC2,color = group_lv1)) +
    geom_point(size = 4) +
    geom_text_repel(
        data = data_plot[data_plot$PC2 < -2,],
        mapping = aes(label = Sample),
        box.padding = 0.5,
        size=6, 
        segment.curvature = 0.5,
        segment.size  = 1,
        force = 100,
        fontface = "italic",
        seed = 42,
        arrow = arrow(length = unit(0.01, "npc")),
        max.overlaps = Inf
    ) +
    scale_color_manual(values = paletteer_d("ggsci::nrc_npg")[c(5,6)]) +
    guides(color = guide_legend(title = 'Cohort Type',override.aes = list(size = 4))) +
    theme_classic() +
    theme(
        plot.margin = margin(l = 0),
        axis.text = element_text(size = 20),
        axis.text.x = element_blank(),
        axis.title = element_blank(),
        legend.position = 'right',
        legend.title = element_text(,size = 20),
        legend.text = element_text(margin = margin(t= 5,b = 5,r = 0,l = 0),size = 20),
        legend.key.width = unit(2, "cm")
    )

In [ ]:
plot(dim_reduction(omicsData = amide_qcrfsc),omicsData = amide_qcrfsc,color_by = "group_lv2")

In [ ]:
plot(dim_reduction(omicsData = amide_qcrfsc),omicsData = amide_qcrfsc,color_by = "SMI_group1")

In [ ]:
plot(dim_reduction(omicsData = amide_qcrfsc),omicsData = amide_qcrfsc,color_by = "SMI_group2")